In [1]:
!pip install openai
!pip3 install tika
!pip install rank_bm25

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=53e30c963bba43bbcd4f834a8694c298b62aea52c37ace820ef37f2e42be8aef
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 불완전판매 모니터링 자동화 - *해피콜*

##### 1. 약관 기반 문서 검색 시스템 구축

In [57]:
from tika import parser

pdf_path = '/content/drive/MyDrive/데이터/part3_ch02_02/보험 약관.pdf'
parsed = parser.from_file(pdf_path)
txt = parsed['content']

passage_len = 1024
corpus = [txt[i:i+passage_len] for i in range(0, len(txt), passage_len)]

In [58]:
len(corpus)

313

In [59]:
corpus[5]

'의무보험을 초과하는 부분에 대해서도  \n\n추가로 가입하는 것이 안전합니다. 또한 의무보험을 포함한 종합보험에 가입한 경우  \n\n교통사고처리특례법 제4조에 의해 형사처벌을 면할 수 있습니다.  \n\n(신호위반, 중앙선침범 등 12대 중과실 제외)\n\n-  대인배상Ⅱ는 대인배상Ⅰ(최고 1억5천만원)을 초과하는 부분을 보상합니다.  대물배상의 경우  \n\n자배법 시행령 제3조 제3항의 규정에 의한 금액(2천만원)까지가 의무보험이며, 이 금액을 초과하는 부분은  \n\n의무보험이 아닙니다.\n\n구분 10일이내 10일초과 최고한도\n\n대인배상Ⅰ 1만원 1일당 4천원 60만원\n\n대물배상 5천원 1일당 2천원 30만원\n\n개인용애니카자동차보험\u2003 9\n\n\n\n1. 청약하신 내용을 확인 하셨나요?\n\n· 피보험자의 성명, 주소, 연령, 연락처 등 인적정보에 관한 사항 \n\n· 차량번호, 차종, 배기량 등 가입차량에 관한 사항 \n\n· 자동차를 운전할 수 있는 자의 범위에 관한 사항 \n\n· 보장종목별 가입금액 \n\n· 보상하는 손해와 보상하지 않는 손해 \n\n· 특별약관 관련사항 등\n\n※  만약, 가입하고자 하는 내용과 다를 경우 계약 담당 설계사나 대리점, 고객콜센터(1588-5114)로 알려주시어  \n\n내용을 정정하시기 바랍니다.\n\n2. 고지하신 사항이 사실과 다름 없나요?\n\n· 다른 보험회사에 가입된 동일 차량에 대한 보험계약 사항 \n\n· 피보험자동차 및 실소유자에 관한 사항 \n\n· 청약서 기재사항 등\n\n※  보험가입시 청약 내용은 사실대로 보험회사에 알려야 하며, 사실과 다르게 알리거나 알리지 않은 경우에는  \n\n보험계약이 해지되거나 보험사고 발생시 보상받지 못할 수도 있습니다.\n\nⅡ. 가입자 유의사항 : 보험계약 체결 시\n\n자배법 제6조 1항의 규정에 따라 자동차보험에 가입한 고객님께  \n\n보험계약 만료 전 2회에 걸쳐 만료 사실을 안내하고 있습니다. \n\n \n\n· 보험기간이 

In [60]:
from rank_bm25 import BM25Okapi

def tokenizer(sent):
  return sent.split(" ")

tokenized_corpus = [tokenizer(doc) for doc in corpus]

bm25 = BM25Okapi(tokenized_corpus)

In [61]:
query = "보험료는 어떻게 낮출 수 있나요?"
tokenized_query = tokenizer(query)

doc_scores = bm25.get_scores(tokenized_query)
doc_scores

passages = bm25.get_top_n(tokenized_query, corpus, n=1)

for p in passages:
  print(p)

 둔 약관이기에 만 26세 미만의  

사람이 운전하던 중 발생한 손해는 보상받으실 수 없습니다.

 -  단, 대인배상Ⅰ은 운전자 연령에 무관하게 보상하여 드립니다. 연령은 사고 발생일 현재  

주민등록상의 만 연령을 기준으로 합니다.

Q.  대물배상도 대인배상Ⅰ과 같이 운전자 범위와 연령 제한을  

위반해도 보상받을 수 있나요?

 A.  대인배상Ⅰ과 함께 대물의무보험은 의무가입 대상이지만 대인배상Ⅰ과 달리  

운전자 범위와 연령 제한을 위반했을 경우 보상이 되지 않습니다.

자동차보험에 대하여 자주나오는 몇 가지 궁금한 내용을 알아봅니다.  

자동차 보험의 내용을 요약하여 설명한 것으로 자세한 내용은 보통약관 및  

특별약관 본문을 참조하시기 바랍니다.

자주하는 질문

Q.  ‘기명 피보험자 1인 한정운전 특별약관’을 가입해도 다른 사람이  

운전할 수 있나요?

 A.  특별약관에서 정하는 운전자의 범위는 기명 피보험자만 가능합니다. 그 외의 사람이 

피보험자동차를 운전 중 발생한 사고는 보상받지 못합니다.(대인배상Ⅰ 만 보상).

26세

36



Q. 보험료는 어떻게 정하나요?

 A.  보험료는 자동차보험 계약 및 사고 자료를 바탕으로 통계적으로 산출됩니다.  

차종, 연령 및 가입금액 등 여러 가지 조건에 따라 보험료가 산출됩니다.

Q. 보험료를 낮추려면 어떻게 해야 하나요?

 A.  운전자의 연령 또는 범위를 한정할 경우 보험료를 줄일 수 있고, Eco 마일리지특약에  

가입하시면 주행거리에 따라 보험료를 할인 받으실 수 있습니다. 무사고의 경우  

보험료가 할인될 수 있으므로 안전운전이 가장 중요합니다.

Q.  보험회사에 알리지 않은 부속품도 보상받을 수 있나요? 

A.  자기차량손해 담보에서 보험회사에 알려주지 않은 자동차의 부속품은  

보상하지 않습니다.

Q. 차량의 부속품을 도난당했는데 보상받을 수 있나요?

 A.  자기차량손해 담보에서 피보험자동차의 일부 부분품, 부속품,  

부속 기계 장치만의 도

##### 문서 기반 문답 생성

In [62]:
from openai import OpenAI

OPENAI_API_KEY= # OpenAI API Key
model_name="gpt-3.5-turbo"

client = OpenAI(
    api_key=OPENAI_API_KEY,
)


In [63]:
from string import Template

query_generation_prompt = Template("""
너는 주어진 문서를 기반으로 사용자가 문서에 대해 제대로 이해하였는지 점검하는 질문을 생성해주는 인공지능이야.

주어진 문서를 기반으로 질문과 질문에 해당하는 답변을 생성해줘. 질문은 문서를 참고하지 않고서도 답변이 가능한 하나의 완결된 독립적인 질문이어야 해

질문의 난이도는 아래 예시와 비슷하게 만들어줘

예시:
{
  질문: "보험금 청구는 어떻게 하나요?"
  답변: "보험금 청구 절차는 우선 온라인 포털 또는 모바일 앱을 통해 신청할 수 있습니다. 필요한 문서(사고 보고서,  의료 기록 등)를 준비한 후, '보험금 청구'  섹션에서 청구 양식을 작성하면 됩니다."
}

{
  질문: "보험금은 언제 받을 수 있나요?"
  답변: "보험금 지급 시간은 청구 절차 완료 후 일반적으로 7 일에서 14 일 사이입니다. 청구한 내용과 문서 검토 상태에 따라 시간이 달라질 수 있습니다.
}

문서: $docs

{
  질문: ...,
  답변: ...
}
""")

In [65]:
def generate_query_base_docs(docs):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": query_generation_prompt.substitute(docs=docs)}
    ]
  )

  return response.choices[0].message.content

In [66]:
print(generate_query_base_docs(corpus[5])) ## random corpus

{
  "질문": "의무보험을 초과하는 부분에 대해서는 추가로 가입하는 것이 안전한 이유는 무엇인가요?",
  "답변": "의무보험을 초과하는 부분에 대해서 추가로 가입하는 것은 추가적인 보상을 받을 수 있는데 도움이 됩니다. 의무보험만으로는 충분한 보상을 받기 어려운 상황에서 추가 가입으로 보호 범위를 확대할 수 있습니다."
}


In [67]:
corpus[5]

'의무보험을 초과하는 부분에 대해서도  \n\n추가로 가입하는 것이 안전합니다. 또한 의무보험을 포함한 종합보험에 가입한 경우  \n\n교통사고처리특례법 제4조에 의해 형사처벌을 면할 수 있습니다.  \n\n(신호위반, 중앙선침범 등 12대 중과실 제외)\n\n-  대인배상Ⅱ는 대인배상Ⅰ(최고 1억5천만원)을 초과하는 부분을 보상합니다.  대물배상의 경우  \n\n자배법 시행령 제3조 제3항의 규정에 의한 금액(2천만원)까지가 의무보험이며, 이 금액을 초과하는 부분은  \n\n의무보험이 아닙니다.\n\n구분 10일이내 10일초과 최고한도\n\n대인배상Ⅰ 1만원 1일당 4천원 60만원\n\n대물배상 5천원 1일당 2천원 30만원\n\n개인용애니카자동차보험\u2003 9\n\n\n\n1. 청약하신 내용을 확인 하셨나요?\n\n· 피보험자의 성명, 주소, 연령, 연락처 등 인적정보에 관한 사항 \n\n· 차량번호, 차종, 배기량 등 가입차량에 관한 사항 \n\n· 자동차를 운전할 수 있는 자의 범위에 관한 사항 \n\n· 보장종목별 가입금액 \n\n· 보상하는 손해와 보상하지 않는 손해 \n\n· 특별약관 관련사항 등\n\n※  만약, 가입하고자 하는 내용과 다를 경우 계약 담당 설계사나 대리점, 고객콜센터(1588-5114)로 알려주시어  \n\n내용을 정정하시기 바랍니다.\n\n2. 고지하신 사항이 사실과 다름 없나요?\n\n· 다른 보험회사에 가입된 동일 차량에 대한 보험계약 사항 \n\n· 피보험자동차 및 실소유자에 관한 사항 \n\n· 청약서 기재사항 등\n\n※  보험가입시 청약 내용은 사실대로 보험회사에 알려야 하며, 사실과 다르게 알리거나 알리지 않은 경우에는  \n\n보험계약이 해지되거나 보험사고 발생시 보상받지 못할 수도 있습니다.\n\nⅡ. 가입자 유의사항 : 보험계약 체결 시\n\n자배법 제6조 1항의 규정에 따라 자동차보험에 가입한 고객님께  \n\n보험계약 만료 전 2회에 걸쳐 만료 사실을 안내하고 있습니다. \n\n \n\n· 보험기간이 

In [68]:
a = generate_query_base_docs(corpus[1]) ## random corpus

In [69]:
import json
a = json.loads(a)

In [70]:
a

{'질문': '자동차 사고 발생 시 어떻게 대처해야 하나요?',
 '답변': '자동차 사고가 발생했을 때, 먼저 안전한 장소로 사고차량을 이동시킨 후 정황증거를 확보해야 합니다. 교통상황이 안전에 위협될 경우 가까운 길가장자리나 안전한 장소로 차량을 이동하고 엔진을 완전히 정지시켜야 합니다. 긴급조치를 취한 뒤에는 경찰서에 신고할지 여부를 고려해야 합니다. 반드시 신고해야 하는 경우와 불필요한 경우를 판단할 때 차량 상태와 교통 안전을 우선 고려해야 합니다.'}

In [71]:
import json
happy_call_test = [{'질문': "해당 약관에 직접 서명하셨나요?",
                    '답변': "네, 제가 직접 싸인했습니다."},
                   {'질문': '보험 약관을 가입 전에 제공받고 확인하셨나요?',
                    '답변': '네, 직원으로부터 전달받고 읽어봤습니다.'}
                   ]
for i in range(10):
  q_a = generate_query_base_docs(corpus[i])
  q_a_json = json.loads(q_a)
  happy_call_test.append(q_a_json)

##### 2. 사용자 답변 평가

In [46]:
from string import Template

evaluate_prompt = Template("""
보험 약관에 질문과 정답이 주어 질 때, 질문에 대한 사용자의 응답이 올바른지 평가해줘.

사용자의 응답이 올바르다면 1과 올바르다고 생각한 이유를 사용자의 응답이 틀렸다면 0과 틀린 이유를 반환해줘.

사용자의 응답이 완전히 정답과 일치하지 않더라도 정답과 같은 의미를 가지고 있다면 올바른 응답이야.

아래와 같은 포멧으로 답변해줘.

{
  정답 여부: {1 or 0},
  이유: ...
}

질문: $q
정답: $a
사용자의 응답: $user_answer
""")

In [72]:
def evaluate(q, a, user_answer):
  response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    response_format={ "type": "json_object" },
    messages=[
      {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
      {"role": "user", "content": evaluate_prompt.substitute(q=q, a=a, user_answer=user_answer)}
    ]
  )

  return response.choices[0].message.content


In [73]:
q = happy_call_test[0]["질문"]
print(q)

해당 약관에 직접 서명하셨나요?


In [74]:
a = happy_call_test[0]["답변"]
print(a)

네, 제가 직접 싸인했습니다.


In [75]:
print(evaluate(q, a, "네, 제가 했어요"))

{
  "정답 여부": 1,
  "이유": "사용자의 응답은 정확한 내용을 전달하고 있으며, 정답과 같은 의미를 가지고 있습니다."
}


In [77]:
print(evaluate(q, a, "직원이 해줬는데요"))

{
  "정답 여부": 0,
  "이유": "사용자의 응답 '직원이 해줬는데요'는 정답과 완전히 일치하지 않습니다. 정답은 '네, 제가 직접 싸인했습니다.'와 다릅니다."
}


##### 3. 해피콜 챗봇 만들어보기

In [78]:
import random

numbers = list(range(11))  # 0부터 10까지의 숫자 리스트
selected_numbers = random.sample(numbers, 5)  # 5개를 중복 없이 무작위로 뽑기

print(selected_numbers)


[7, 0, 9, 2, 8]


In [79]:
import random

class HappyCallBot():
  def __init__(self, q_a_list):
    self.q_a_list = q_a_list

    self.test_set = None
    self.num_solved = 0
    self.num_wrong = 0

  def build_test_set(self, num_test_sample=5):
    ## test set 만들기
    # num_test_sample = min(num_test_sample, len(self.q_a_list))
    # numbers = list(range(len(self.q_a_list)))
    # selected_numbers = random.sample(numbers, num_test_sample)
    # test_set = new_array = [self.q_a_list[i] for i in selected_numbers]

    self.test_set = self.q_a_list
    self.num_solved = 0
    self.num_wrong = 0

  def query(self):
    id = self.num_solved + self.num_wrong
    if id == len(self.test_set):
      print("테스트가 종료 되었습니다.")
      print(f"전체 {len(self.test_set)} 질문 중 올바른 응답은 {self.num_solved} 였습니다.")
    q = self.test_set[id]["질문"]
    print(q)

  def answer(self, ans):
    id = self.num_solved + self.num_wrong
    q = self.test_set[id]["질문"]
    a = self.test_set[id]["답변"]
    result = evaluate(q, a, ans)
    result = json.loads(result)

    if result["정답 여부"] == 1:
      self.num_solved += 1
    else :
      self.num_wrong += 1

    return result

  def evaluate(q, a, user_answer):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": evaluate_prompt.substitute(q=q, a=a, user_answer=user_answer)}
      ]
    )

    eval = json.loads(response.choices[0].message.content)
    return eval


  def start_test(self, num_test_sample=5):
    self.build_test_set(num_test_sample)



In [80]:
happyCallBot =  HappyCallBot(happy_call_test)

In [81]:
happyCallBot.start_test()

In [109]:
happyCallBot.query()

보험금 청구를 했는데 지급이 지연되는 경우 대응 방법은 무엇인가요?


In [110]:
happyCallBot.answer(ans="트렁크에서 정차 신호를 표시합니다.")

{'정답 여부': 0, '이유': '사용자의 응답이 정답과 전혀 다른 내용이므로 오답입니다.'}

In [111]:
print(happyCallBot.num_solved)
print(happyCallBot.num_wrong)

1
11


##### 4. 해피콜 보이스 봇 만들기

In [112]:
import os
import random
from IPython.display import Audio, display


class HappyCallVoiceBot():
  def __init__(self, cache_path, q_a_list):
    self.cache_path = cache_path
    self.cache_id = 10
    self.q_a_list = q_a_list

    self.test_set = None
    self.num_solved = 0
    self.num_wrong = 0

  def build_test_set(self, num_test_sample=5):
    ## test set 만들기
    num_test_sample = min(num_test_sample, len(self.q_a_list))
    numbers = list(range(len(self.q_a_list)))
    selected_numbers = random.sample(numbers, num_test_sample)
    test_set = new_array = [self.q_a_list[i] for i in selected_numbers]

    self.test_set = test_set
    self.num_solved = 0
    self.num_wrong = 0

  def query(self):
    id = self.num_solved + self.num_wrong

    if id == len(self.test_set):
      print("테스트가 종료 되었습니다.")
      print(f"전체 {len(self.test_set)} 질문 중 올바른 응답은 {self.num_solved} 였습니다.")
      return
    q = self.test_set[id]["질문"]
    print(q)

  def answer(self, ans):
    id = self.num_solved + self.num_wrong
    q = self.test_set[id]["질문"]
    a = self.test_set[id]["답변"]
    result = evaluate(q, a, ans)
    result = json.loads(result)

    if result["정답 여부"] == 1:
      self.num_solved += 1
    else :
      self.num_wrong += 1

    self.tts(result["이유"])

  def evaluate(q, a, user_answer):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      response_format={ "type": "json_object" },
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": evaluate_prompt.substitute(q=q, a=a, user_answer=user_answer)}
      ]
    )

    eval = response.choices[0].message.content
    return eval


  def start_test(self, num_test_sample=5):
    self.build_test_set(num_test_sample)


  def tts(self, txt):
    save_path = os.path.join(self.cache_path, f"{self.cache_id}.mp3")
    response = client.audio.speech.create(
      model="tts-1",
      voice="echo",
      input=txt
    )
    is_save = response.stream_to_file(save_path)

    display(Audio(save_path, autoplay=True))
    self.cache_id += 1

    return save_path

In [113]:
cache_path = "/content/drive/MyDrive/데이터/part3_ch02_02/cache"

happyCallVoiceBot = HappyCallVoiceBot(cache_path, happy_call_test)

In [114]:
happyCallVoiceBot.start_test()

In [117]:
happyCallVoiceBot.query()

자동차 사고 시 안전조치는 무엇이 있나요?


In [118]:
happyCallVoiceBot.answer("자동차를 갓길로 옮깁니다")

<ipython-input-112-b8544fa5b358>:76: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


In [ ]:
happyCallVoiceBot.query()

차량 사고로 인해 부상을 입었을 때 어떻게 보험금을 청구할 수 있나요?


In [ ]:
happyCallVoiceBot.answer("병원에 가서 보험사 직원과 협의하고, 치료 후 진단서를 가지고 보험을 청구하면 됩니다")

<ipython-input-79-a6fcdec0129a>:75: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


In [ ]:
happyCallVoiceBot.query()

후방하자 발생시 어떤 작업을 검토하나요?


In [ ]:
happyCallVoiceBot.answer("잘 모르겠어요")

<ipython-input-79-a6fcdec0129a>:75: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


In [ ]:
happyCallVoiceBot.query()

자동차 사고 시 어떻게 대처해아 할까요?


In [ ]:
happyCallVoiceBot.answer("안전한 장소로 차량을 이동 시키고 정황 증거를 확보해야 합니다.")

<ipython-input-79-a6fcdec0129a>:75: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


In [ ]:
happyCallVoiceBot.query()

의료기관의 진단서에 근거하여 치료비가 인정되는 경우는 어떤 경우인가요?


In [ ]:
happyCallVoiceBot.answer("진단서에 근거하여 치료를 받을 때 입니다.")

<ipython-input-79-a6fcdec0129a>:75: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  is_save = response.stream_to_file(save_path)


In [ ]:
happyCallVoiceBot.test_set

[{'질문': '피보험자는 누구인가요?',
  '답변': '피보험자는 보상을 청구할 수 있는 사람으로, 보험증권에 기재된 기명 피보험자뿐만 아니라 기명 피보험자와 함께 살거나 살림을 같이하는 친족, 기명 피보험자의 승낙을 받은 사람 등이 해당됩니다.'},
 {'질문': '차량 사고로 인해 부상을 입었을 때 어떻게 보험금을 청구할 수 있나요?',
  '답변': "부상을 입은 경우 병원으로 후송되어 진찰 및 치료방법을 결정하고 보상직원과 협의를 하게 됩니다. 치료가 완료된 후 보험금을 청구할 수 있으며, 청구 절차는 온라인 포털 또는 모바일 앱을 통해 신청할 수 있습니다. 필요한 문서(의료 기록, 치료비 영수증 등)를 제출한 후 '보험금 청구' 섹션에서 청구 양식을 작성하면 됩니다."},
 {'질문': '후방하자 발생시 어떤 작업을 검토하나요?',
  '답변': '후방하자 발생시에는 범퍼와 기타 외장부품에 대한 작업을 검토합니다. 범퍼의 경우 1유형, 2유형, 3유형에 따라 서로 다른 수리방법을 적용하며, 기타 외장부품의 경우에도 1유형, 2유형, 3유형에 따라 다른 수리방법을 사용합니다.'},
 {'질문': '자동차 사고 시 어떻게 대처해아 할까요?',
  '답변': '자동차 사고가 발생하면 다음과 같은 절차로 대처해야 합니다. 먼저, 도로상 표시(스프레이, 페인트 등)와 목격자 확보(인적사항, 연락처)를 해야 합니다. 안전한 장소로 사고차량을 이동시킨 후 정황증거를 확보하고 교통상황이 위협될 경우 위험방지를 위해 가까운 길가장자리나 안전한 장소로 차량을 이동하고 엔진을 완전히 정지시켜야 합니다. 이후에는 긴급조치를 취한 후 경찰서에 신고해야 합니다. 만약 차량 파손만 있고 위험방지와 소통이 원활하게 이루어졌을 때에는 신고할 필요는 없습니다.'},
 {'질문': '의료기관의 진단서에 근거하여 치료비가 인정되는 경우는 어떤 경우인가요?',
  '답변': '의료기관의 진단서에 근거하여 치료비가 인정되는 경우는 치료를 받고자 할 경우입니다.'}]